<a href="https://colab.research.google.com/github/lsgotti/TCC_Python/blob/main/oversampling_crosval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#carrega imports e instala o scikit-multiflow
!pip install -U scikit-multiflow
# Imports
from skmultiflow.data import FileStream
from skmultiflow.data import SEAGenerator
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.bayes import NaiveBayes
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import LearnPPNSEClassifier
from skmultiflow.drift_detection import ADWIN
from skmultiflow.meta import OzaBaggingClassifier

from sklearn.linear_model import SGDClassifier

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn import svm
import pandas as pd
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from skmultiflow.lazy import KNNClassifier

from google.colab import files

import numpy as np
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
import pandas as pd
import io
from google.colab import files
from skmultiflow.trees import HoeffdingTree
from skmultiflow.meta import AdaptiveRandomForestClassifier
from datetime import datetime, date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


In [ ]:
#segunda parte
#importa arquivo para teste
uploaded = files.upload()
#Importa o arquivo para o colab
import io

#abre o arquivo no dataframe
df = pd.read_csv(io.BytesIO(uploaded['tomcat3.csv']))

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:25f}'.format})

#Etapa de upsampling

from sklearn.utils import resample

#Divide o dataframe em codigos 0 e 1, apenas o 1 sera usado

m=df.code==0
df_code0=df[m]
df_code1=df[~m]

print(df_code1.contains_bug.value_counts())

# Separa classes minoritarias e majoritarias
df_majority = df_code1[df_code1.contains_bug==0]
df_minority = df_code1[df_code1.contains_bug==1]


# Upsample de classe minoritaria
df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)


# Combina classe majoritaria e minoritaria
df_upsampled_partial = pd.concat([df_majority, df_minority_upsampled])

print(df_upsampled_partial.contains_bug.value_counts())
df_upsampled = pd.concat([df_upsampled_partial, df_code0])
print(df_upsampled.contains_bug.value_counts())

#embaralha o dataframe
df_upsampled = df_upsampled.sample(frac=1)

df_traindata = df_upsampled[['contains_bug']]
df_upsampled.drop('contains_bug', inplace=True, axis=1)
df_upsampled.drop('code', inplace=True, axis=1)
df_upsampled.drop('days_to_first_fix', inplace=True, axis=1)
df_upsampled.drop('repository_name', inplace=True, axis=1)

df_traindata2 = df_traindata.to_numpy().reshape(df_upsampled.shape[0],)
df_upsampled2 = df_upsampled.to_numpy()
df_upsampled2.shape, df_traindata2.shape

0    8066
1    2326
Name: contains_bug, dtype: int64
0    8066
1    8066
Name: contains_bug, dtype: int64
0    14483
1    10392
Name: contains_bug, dtype: int64


((24875, 16), (24875,))

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import math
import numpy as np
import copy as cp
import matplotlib.pyplot as plt

import seaborn as sns
from typing import Tuple
from sklearn.metrics import confusion_matrix

kfold = KFold(n_splits=10, random_state=42, shuffle=True)

model_ = DecisionTreeClassifier()

actual_classes = np.empty([0], dtype=int)
predicted_classes = np.empty([0], dtype=int)

rec1=  0
rec0 = 0
gmean = 0
recall0list = []
recall1list = []
gmeanList = []

for train_ndx, test_ndx in kfold.split(df_upsampled2):
  train_X, train_y, test_X, test_y = df_upsampled2[train_ndx], df_traindata2[train_ndx], df_upsampled2[test_ndx], df_traindata2[test_ndx]
  actual_classes = np.empty([0], dtype=int)
  predicted_classes = np.empty([0], dtype=int)

  actual_classes = np.append(actual_classes, test_y)

  model_.fit(train_X, train_y)
  predicted_classes = np.append(predicted_classes, model_.predict(test_X))

  conf_mat = confusion_matrix(actual_classes,predicted_classes)

  print(conf_mat)
  currentrec1 = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
  currentrec0 = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
  currentgmean = math.sqrt((conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])) * (conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])))
  recall0list.append(currentrec0)
  recall1list.append(currentrec1)
  gmeanList.append(currentgmean)
  rec1 += conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
  rec0 += conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
  gmean += math.sqrt((conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])) * (conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])))

  print(currentrec0, currentrec1, currentgmean)

rec1 = rec1/10
rec0 = rec0/10
gmean = gmean/10
recall0list.append(rec0)
recall1list.append(rec1)
gmeanList.append(gmean)

print("rec0, rec1 e gmean")
print(rec0, rec1, gmean)

results = pd.DataFrame(
    {'recall0': recall0list,
     'recall1': recall1list,
     'gmean': gmeanList
    })

results.to_csv('results.csv')
files.download('results.csv')

[[1323  136]
 [ 140  889]]
0.8673170731707317 0.9043062200956937 0.8856185544936852
[[1302  147]
 [ 104  935]]
0.8641404805914972 0.9260312944523471 0.8945507966744076
[[1290  166]
 [ 128  904]]
0.8448598130841122 0.9097320169252469 0.8766960829022201
[[1297  168]
 [ 112  911]]
0.8443002780352178 0.9205110007097232 0.8815824940603664
[[1322  151]
 [ 102  913]]
0.8580827067669173 0.9283707865168539 0.8925351070841494
[[1255  160]
 [ 111  961]]
0.8572702943800179 0.9187408491947291 0.8874735141107666
[[1290  142]
 [ 121  934]]
0.8680297397769516 0.9142452161587526 0.890837828717778
[[1324  145]
 [ 115  903]]
0.8616412213740458 0.9200833912439194 0.8903829384020028
[[1242  145]
 [ 113  987]]
0.8719081272084805 0.9166051660516605 0.8939773451949001
[[1334  144]
 [ 108  901]]
0.8622009569377991 0.9251040221914009 0.8930988597016754
rec0, rec1 e gmean
0.8599750691325772 0.9183729963540326 0.8886753521341951


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(conf_mat)
import math
rec1=  0
rec0 = 0
gmean = 0
rec1 += conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
rec0 += conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
gmean += math.sqrt(rec0 * rec1)

print(rec0, rec1, gmean)

[[15470  1882]
 [ 1190  6880]]
0.7852088564254737 0.9285714285714286 0.8538867076714216
